In [ ]:
# Imports 

import numpy as np
import scipy.io as sio
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import torch
from torchvision import transforms
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA
from PIL import Image

from sklearn.linear_model import LinearRegression
import numpy as np

In [ ]:
#Loading the data

data = sio.loadmat('dataset')
x = data['data']
y = data['label']
y = y.reshape(-1,1)
y = np.array(y)
x = x.reshape(51,51,51,5900)
x = np.transpose(x,(3,0,1,2))

xx_bin = x[:,0,:,:]
xy_bin = x[:,:,0,:]
xz_bin = x[:,:,:,0]

In [ ]:
# Performing Interpolation

transform = transforms.Compose([
                transforms.Resize(224 ,interpolation=transforms.InterpolationMode.BICUBIC),
                transforms.ToTensor(), 
                 ])

# Working with the Base model here, can change. 

dinov2_vitb14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitb14')

# Making features

def make_features(x_bin):

    x_t = torch.Tensor()
    for img in x_bin:
        img = Image.fromarray(np.uint8(img*255)).convert('RGB')
        img_t = transform(img)
        img_t = img_t[None,:,:,:]
        x_t = torch.cat((x_t,img_t),0)

    return dinov2_vitb14(x_t).detach().numpy()

def batch_make_features(xx_bin,n_batch=590):
    for ii in range(10):
        xx_bin_i = xx_bin[ii*n_batch:(ii+1)*n_batch,:,:]
        xx_i = make_features(xx_bin_i)  
        if ii == 0:
            xx = xx_i
        else: 
            xx = np.vstack((xx,xx_i))
        print('done with round ', ii)
        
    return xx

In [ ]:
xx = batch_make_features(xx_bin)
xy = batch_make_features(xy_bin)
xz = batch_make_features(xz_bin)

In [ ]:
np.save('xx_interpolate.npy',xx)
np.save('xy_interpolate.npy',xy)
np.save('xz_interpolate.npy',xz)